## HW4. Контейнеризация с Docker
Автор: *Насибулин Алексей Алексеевич*

Группа: *М08-401НД*

Дата: *17.11.25*

In [1]:
!docker -v

Docker version 28.5.1, build e180ab8


In [40]:
!docker build --progress=plain --tag=test-app:v1 simple_app

#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 628B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.12.3-alpine3.20
#2 DONE 1.7s

#3 [internal] load .dockerignore
#3 transferring context: 71B done
#3 DONE 0.0s

#4 [builder 1/3] FROM docker.io/library/python:3.12.3-alpine3.20@sha256:32385e61c3414ffa5a6dbf52feace89f758ad68709a48d376d56a0232162665a
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 442B done
#5 DONE 0.0s

#6 [stage-1 3/4] RUN --mount=type=bind,from=builder,source=wheels,target=/wheels     pip install --no-index --find-links /wheels -r requirements.txt
#6 CACHED

#7 [builder 2/3] COPY requirements.txt .
#7 CACHED

#8 [builder 3/3] RUN --mount=type=cache,id=pip-cache,target=/root/.cache/pip     pip wheel -r requirements.txt -w wheels/
#8 CACHED

#9 [stage-1 4/4] COPY . .
#9 CACHED

#10 exporting to image
#10 exporting layers done
#10 w

In [25]:
!docker image ls | grep test-app

test-app                         v1                             c19567b1b637   15 seconds ago       66.1MB


In [36]:
!(cd simple_app; BUILDKIT_PROGRESS=plain docker compose --profile dev up --watch)

[+] Running 1/2
 ✔ Network simple_app_default      Created                                 0.0s 
 ⠋ Container simple_app-app-dev-1  Cre...                                  0.1s 
[+] Running 2/2
 ✔ Network simple_app_default      Created                                 0.0s 
 ✔ Container simple_app-app-dev-1  Cre...                                  0.1s 
78        ⦿ Watch enabled
78

7w Disable Watch8Attaching to app-dev-1
78app-dev-1  |  * Serving Flask app 'app'
78

7w Disable Watch878app-dev-1  |  * Debug mode: off
78

7w Disable Watch878app-dev-1  | WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
78

7w Disable Watch878app-dev-1  |  * Running on all addresses (0.0.0.0)
78

7w Disable Watch878app-dev-1  |  * Running on http://127.0.0.1:5000
78

7w Disable Watch878app-dev-1  |  * Running on http://172.19.0.2:5000
78

7w Disable Watch878app-dev-1  | Press CTRL+C to quit
7

In [39]:
!(cd simple_app; BUILDKIT_PROGRESS=plain docker compose --profile dev down)

[+] Running 1/2
 ✔ Container simple_app-app-dev-1  Rem...                                  0.0s 
 ⠋ Network simple_app_default      Removin...                              0.1s 
[+] Running 1/2
 ✔ Container simple_app-app-dev-1  Rem...                                  0.0s 
 ⠙ Network simple_app_default      Removin...                              0.2s 
[+] Running 2/2
 ✔ Container simple_app-app-dev-1  Rem...                                  0.0s 
 ✔ Network simple_app_default      Removed                                 0.2s 


In [68]:
!mkdir -p registry/certs
!openssl req -x509 -newkey rsa:4096 -keyout registry/certs/registry.key -out registry/certs/registry.crt -sha256 -days 3650 -nodes -subj "/C=XX/ST=StateName/L=CityName/O=CompanyName/OU=CompanySectionName/CN=localhost"

......+.......+..+.............+............+..+...+.+..+....+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.+.+..+....+......+.....+...+....+............+........+.+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.......+...+...+......+.....+..........+...+...+..............+......+.........+..........+.........+.....+.......+........+......+..................+.+.........+..+......+.........+....+.................+....+........+...+...+.+..................+..+..........+......+.........+..................+..+.............+.....+.+..+............+...+............+..........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
..+...+..+......+.+...........+.......+...+..+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*...+...........+......+.+...+.....+.+.....+.............+...+..+......+......+...............+.+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.....+.....+.+.....+....+......+.........+....

In [78]:
!docker run -d --restart=always --name registry \
  -v $(pwd)/registry/data:/var/lib/registry \
  -v $(pwd)/registry/certs:/certs \
  -e REGISTRY_HTTP_ADDR=0.0.0.0:443 \
  -e REGISTRY_HTTP_TLS_CERTIFICATE=/certs/registry.crt \
  -e REGISTRY_HTTP_TLS_KEY=/certs/registry.key \
  -p 443:443 \
  registry:3

a2a380614092eb8a0ab3214a2ad8fd466e7c7aca506c2ab6beae1cadd90b0c97


In [79]:
!docker tag test-app:v1 localhost/test-app:v1
!docker push localhost/test-app:v1
# Remove image tag to trigger registry usage
!docker image rm localhost/test-app:v1

The push refers to repository [localhost/test-app]

d79c0189: Preparing 
ee7f3bce: Preparing 
4b11d755: Preparing 
a573bcd3: Preparing 
1f60a500: Preparing 
fc806acc: Preparing 
acde20ab: Preparing 
bcb26af5: Layer already exists v1: digest: sha256:5a730f33bd97d1cd09e4fb5f1ecd20417a9e9ae5e5db3e0add7d3532959e8eff size: 1993
Untagged: localhost/test-app:v1
Untagged: localhost/test-app@sha256:5a730f33bd97d1cd09e4fb5f1ecd20417a9e9ae5e5db3e0add7d3532959e8eff


In [82]:
%%writefile main.tf
terraform {
  required_providers {
    docker = {
      source  = "kreuzwerker/docker"
      version = "3.9.0"
    }
  }
}

provider "docker" {
  host = "unix:///var/run/docker.sock"
}

data "docker_registry_image" "my_app_image" {
  name = "localhost/test-app:v1"
  insecure_skip_verify = true
}

resource "docker_image" "app" {
  name = data.docker_registry_image.my_app_image.name
  pull_triggers = [data.docker_registry_image.my_app_image.sha256_digest]
}

resource "docker_container" "my_app" { 
    name  = "my_app"
    image = docker_image.app.name
    ports {
        internal = 5000
        external = 4000
    }
}

Overwriting main.tf


In [59]:
!tofu init


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of kreuzwerker/docker from the dependency lock file
- Using previously-installed kreuzwerker/docker v3.9.0

OpenTofu has been successfully initialized!

You may now begin working with OpenTofu. Try running "tofu plan" to see
any changes that are required for your infrastructure. All OpenTofu commands
should now work.

If you ever set or change modules or backend configuration for OpenTofu,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [83]:
!tofu validate
!tofu plan

Success! The configuration is valid.
data.docker_registry_image.my_app_image: Reading...
data.docker_registry_image.my_app_image: Read complete after 1s [id=sha256:5a730f33bd97d1cd09e4fb5f1ecd20417a9e9ae5e5db3e0add7d3532959e8eff]

OpenTofu used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

OpenTofu will perform the following actions:

  # docker_container.my_app will be created
  + resource "docker_container" "my_app" {
      + attach                                      = false
      + bridge                                      = (known after apply)
      + command                                     = (known after apply)
      + container_logs                              = (known after apply)
      + container_read_refresh_timeout_milliseconds = 15000
      + entrypoint                                  = (known after apply)
      + env                                         = (known after app

In [84]:
!tofu apply -auto-approve

data.docker_registry_image.my_app_image: Reading...
data.docker_registry_image.my_app_image: Read complete after 0s [id=sha256:5a730f33bd97d1cd09e4fb5f1ecd20417a9e9ae5e5db3e0add7d3532959e8eff]

OpenTofu used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

OpenTofu will perform the following actions:

  # docker_container.my_app will be created
  + resource "docker_container" "my_app" {
      + attach                                      = false
      + bridge                                      = (known after apply)
      + command                                     = (known after apply)
      + container_logs                              = (known after apply)
      + container_read_refresh_timeout_milliseconds = 15000
      + entrypoint                                  = (known after apply)
      + env                                         = (known after apply)
      + exit_code                

In [85]:
!tofu destroy -auto-approve

data.docker_registry_image.my_app_image: Reading...
data.docker_registry_image.my_app_image: Read complete after 0s [id=sha256:5a730f33bd97d1cd09e4fb5f1ecd20417a9e9ae5e5db3e0add7d3532959e8eff]
docker_image.app: Refreshing state... [id=sha256:5113df76b0913a986522af3a56ed3c105325d1071cc53dd53461c03bc9225f24localhost/test-app:v1]
docker_container.my_app: Refreshing state... [id=782bf44ec1dde2cd48b43e775a18d6ad62e69029d3c2f899736d2e07bc22e648]

OpenTofu used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

OpenTofu will perform the following actions:

  # docker_container.my_app will be destroyed
  - resource "docker_container" "my_app" {
      - attach                                      = false -> null
      - command                                     = [
          - "python",
          - "app.py",
        ] -> null
      - container_read_refresh_timeout_milliseconds = 15000 -> null
      - cpu_sha